In [1]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
from langchain_core.tools import tool

In [4]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplica dos números enteros."""
    return first_int * second_int

In [5]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [7]:
from langchain.tools.render import render_text_description

In [8]:
rendered_tools = render_text_description([multiply])
rendered_tools

'multiply: multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.'

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [23]:
system_prompt = f"""Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:

{rendered_tools}

Dada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.
"""

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), 
        ("user", "{input}")
    ]
)

In [25]:
from langchain_core.output_parsers import JsonOutputParser

In [20]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [32]:
chain = prompt | llm | JsonOutputParser()
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:\n\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.\n\nDada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002024876BBE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002024879C280>, model_name='

In [27]:
respuesta =chain.invoke({"input": "¿Cuanto es trece veces 4?"})
respuesta

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

# Invocando la Herramienta

In [28]:
from operator import itemgetter

In [30]:
chain = prompt | llm | JsonOutputParser() | itemgetter("arguments") | multiply
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Usted es un asistente que tiene acceso al siguiente conjunto de herramientas. Estos son los nombres y descripciones de cada herramienta:\n\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.\n\nDada la entrada del usuario, devuelva únicamente el nombre y los argumentos de la herramienta a utilizar en formato JSON válido. La respuesta debe ser un JSON con las claves 'name' y 'arguments', donde 'arguments' es un diccionario con los argumentos de la herramienta. No incluya ninguna otra explicación en la respuesta.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002024876BBE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002024879C280>, model_name='

In [31]:
resultado = chain.invoke({"input": "¿Cuanto es 30 veces cuatro"})
resultado

120